In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('//content/gdrive/My Drive/Colab Notebooks/')

In [0]:
#export
from exp.nb_11 import *

### serializing the model

In [0]:
path = datasets.untar_data(datasets.URLs.IMAGEWOOF_160)

In [0]:
size = 128
bs = 64

tfms = [make_rgb, RandomResizedCrop(size, scale=(0.35,1)), np_to_float, PilRandomFlip()]
val_tfms = [make_rgb, CenterCrop(size), np_to_float]
il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())
ll.valid.x.tfms = val_tfms
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=8)

In [5]:
len(il)

12954

In [0]:
loss_func = LabelSmoothingCrossEntropy()
opt_func = adam_opt(mom=0.9,mom_sqr=0.99,eps=1e-6,wd=1e-2)
learn = cnn_learner(xresnet18,data,loss_func,opt_func,norm=norm_imagenette)

In [0]:
def sched_1cycle(lr,pct_start=0.3,mom_start=0.95,mom_mid=0.85,mom_end=0.95):
  phases = create_phases(pct_start)
  sched_lr = combine_scheds(phases, cos_1cycle_anneal(lr/10.,lr,lr/1e5))
  sched_mom = combine_scheds(phases,cos_1cycle_anneal(mom_start,mom_mid,mom_end))
  return [ParamScheduler('lr',sched_lr),ParamScheduler('mom',sched_mom)]

In [0]:
lr = 3e-3
pct_start = 0.5
cbsched = sched_1cycle(lr,pct_start)

In [9]:
learn.fit(40,cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.122712,0.250361,2.106088,0.244000,00:43
1,1.965386,0.335796,1.942081,0.344000,00:43
2,1.839683,0.398908,1.981215,0.368000,00:43
3,1.755532,0.440340,2.469433,0.276000,00:43
4,1.679809,0.476554,1.795734,0.394000,00:43
5,1.640973,0.502248,1.852748,0.382000,00:43
6,1.609613,0.512928,1.761320,0.416000,00:43
7,1.579982,0.532520,1.811412,0.422000,00:43
8,1.537742,0.545126,1.816030,0.422000,00:43
9,1.487730,0.570660,1.567914,0.532000,00:43


In [10]:
st = learn.model.state_dict()
', '.join(st.keys())

'0.0.weight, 0.1.weight, 0.1.bias, 0.1.running_mean, 0.1.running_var, 0.1.num_batches_tracked, 1.0.weight, 1.1.weight, 1.1.bias, 1.1.running_mean, 1.1.running_var, 1.1.num_batches_tracked, 2.0.weight, 2.1.weight, 2.1.bias, 2.1.running_mean, 2.1.running_var, 2.1.num_batches_tracked, 4.0.convs.0.0.weight, 4.0.convs.0.1.weight, 4.0.convs.0.1.bias, 4.0.convs.0.1.running_mean, 4.0.convs.0.1.running_var, 4.0.convs.0.1.num_batches_tracked, 4.0.convs.1.0.weight, 4.0.convs.1.1.weight, 4.0.convs.1.1.bias, 4.0.convs.1.1.running_mean, 4.0.convs.1.1.running_var, 4.0.convs.1.1.num_batches_tracked, 4.1.convs.0.0.weight, 4.1.convs.0.1.weight, 4.1.convs.0.1.bias, 4.1.convs.0.1.running_mean, 4.1.convs.0.1.running_var, 4.1.convs.0.1.num_batches_tracked, 4.1.convs.1.0.weight, 4.1.convs.1.1.weight, 4.1.convs.1.1.bias, 4.1.convs.1.1.running_mean, 4.1.convs.1.1.running_var, 4.1.convs.1.1.num_batches_tracked, 5.0.convs.0.0.weight, 5.0.convs.0.1.weight, 5.0.convs.0.1.bias, 5.0.convs.0.1.running_mean, 5.0.convs

In [11]:
st['10.bias']

tensor([ 0.0157, -0.0065, -0.0081, -0.0167, -0.0158,  0.0247, -0.0733,  0.0120,
         0.0066,  0.0444], device='cuda:0')

In [0]:
mdl_path=path/'models'
mdl_path.mkdir(exist_ok=True)

In [0]:
torch.save(st,mdl_path/'iw5')

### pets

In [0]:
pets = datasets.untar_data(datasets.URLs.PETS)

In [21]:
pets.ls()

[PosixPath('/root/.fastai/data/oxford-iiit-pet/annotations'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/images')]

In [0]:
pets_path=pets/'images'

In [0]:
il = ImageList.from_files(pets_path,tfms=tfms)

In [24]:
il

ImageList (7390 items)
[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/english_setter_61.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/basset_hound_111.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/scottish_terrier_7.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_222.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_67.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/english_setter_154.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/saint_bernard_46.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/yorkshire_terrier_120.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/beagle_164.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Siamese_238.jpg')...]
Path: /root/.fastai/data/oxford-iiit-pet/images

In [0]:
#export
def random_splitter(fn,p_valid): return random.random() < p_valid

In [0]:
random.seed(42)

In [0]:
sd = SplitData.split_by_func(il,partial(random_splitter,p_valid=0.1))

In [28]:
sd

SplitData
Train: ImageList (6667 items)
[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/english_setter_61.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/scottish_terrier_7.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_222.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_67.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/english_setter_154.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/saint_bernard_46.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/beagle_164.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/keeshond_89.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Birman_109.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/havanese_148.jpg')...]
Path: /root/.fastai/data/oxford-iiit-pet/images
Valid: ImageList (723 items)
[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/basset_hound_111.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/im

In [30]:
n = il.items[0].name; n

'english_setter_61.jpg'

In [31]:
re.findall(r'^(.*)_\d+.jpg$',n)[0]

'english_setter'

In [0]:
def pet_labeler(fn): return re.findall(r'^(.*)_\d+.jpg$',fn.name)[0]

In [0]:
proc = CategoryProcessor()

In [0]:
ll = label_by_func(sd,pet_labeler,proc_y=proc)

In [35]:
', '.join(proc.vocab)

'english_setter, scottish_terrier, Egyptian_Mau, saint_bernard, beagle, keeshond, Birman, havanese, Siamese, newfoundland, american_pit_bull_terrier, Bombay, miniature_pinscher, Ragdoll, american_bulldog, Russian_Blue, basset_hound, great_pyrenees, Bengal, leonberger, Persian, pomeranian, British_Shorthair, english_cocker_spaniel, Abyssinian, Sphynx, german_shorthaired, wheaten_terrier, samoyed, chihuahua, boxer, pug, staffordshire_bull_terrier, japanese_chin, shiba_inu, Maine_Coon, yorkshire_terrier'

In [0]:
ll.valid.x.tfms = val_tfms

In [0]:
c_out=len(proc.vocab)

In [0]:
data = ll.to_databunch(bs,c_in=3,c_out=c_out, num_workers=8)

In [0]:
learn = cnn_learner(xresnet18,data,loss_func,opt_func,norm=norm_imagenette)

In [40]:
learn.fit(5,cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.473916,0.088946,3.462662,0.084371,00:37
1,3.292235,0.136343,3.582549,0.081604,00:37
2,3.080751,0.191240,3.211836,0.146611,00:37
3,2.774271,0.289636,2.750294,0.273859,00:37
4,2.476247,0.389081,2.512551,0.359613,00:37


### custom head

In [0]:
learn = cnn_learner(xresnet18,data,loss_func,opt_func,c_out=10,norm=norm_imagenette)

In [0]:
st = torch.load(mdl_path/'iw5')

In [0]:
m = learn.model

In [44]:
m.load_state_dict(st)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])